**Autograd** : Auto grad is a core component of PyTorch that provides automatic differentiation for tensor operations. It enables gradient computation, which is essential for training ML models using optimization algorithms like gradient descent.

In [1]:
def dy_dx(x):
  return 2*x

In [2]:
dy_dx(3)

6

In [3]:
import torch

In [4]:
x = torch.tensor(3.0, requires_grad=True)

In [5]:
y = x**2

In [6]:
x

tensor(3., requires_grad=True)

In [7]:
y

tensor(9., grad_fn=<PowBackward0>)

In [8]:
y.backward()

In [9]:
x.grad

tensor(6.)

In [10]:
import math

def dz_dx(x):
    return 2 * x * math.cos(x**2)

In [11]:
dz_dx(4)

-7.661275842587077

In [12]:
x = torch.tensor(4.0, requires_grad=True)

In [13]:
y = x ** 2

In [14]:
z = torch.sin(y)

In [15]:
x

tensor(4., requires_grad=True)

In [16]:
y

tensor(16., grad_fn=<PowBackward0>)

In [17]:
z

tensor(-0.2879, grad_fn=<SinBackward0>)

In [18]:
z.backward()

In [19]:
x.grad

tensor(-7.6613)

In [20]:
y.grad

<ipython-input-20-10b3a7061f6d>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  y.grad


**Manually**

In [21]:
import torch

# Inputs
x = torch.tensor(6.7)  # Input feature
y = torch.tensor(0.0)  # True label (binary)

w = torch.tensor(1.0)  # Weight
b = torch.tensor(0.0)  # Bias

In [22]:
# Binary Cross-Entropy Loss for scalar
def binary_cross_entropy_loss(prediction, target):
    epsilon = 1e-8  # To prevent log(0)
    prediction = torch.clamp(prediction, epsilon, 1 - epsilon)
    return -(target * torch.log(prediction) + (1 - target) * torch.log(1 - prediction))

In [23]:
# Forward pass
z = w * x + b  # Weighted sum (linear part)
y_pred = torch.sigmoid(z)  # Predicted probability

# Compute binary cross-entropy loss
loss = binary_cross_entropy_loss(y_pred, y)

In [24]:
loss

tensor(6.7012)

In [25]:
# Derivatives:
# 1. dL/d(y_pred): Loss with respect to the prediction (y_pred)
dloss_dy_pred = (y_pred - y)/(y_pred*(1-y_pred))

# 2. dy_pred/dz: Prediction (y_pred) with respect to z (sigmoid derivative)
dy_pred_dz = y_pred * (1 - y_pred)

# 3. dz/dw and dz/db: z with respect to w and b
dz_dw = x  # dz/dw = x
dz_db = 1  # dz/db = 1 (bias contributes directly to z)

dL_dw = dloss_dy_pred * dy_pred_dz * dz_dw
dL_db = dloss_dy_pred * dy_pred_dz * dz_db

In [26]:
print(f"Manual Gradient of loss w.r.t weight (dw): {dL_dw}")
print(f"Manual Gradient of loss w.r.t bias (db): {dL_db}")

Manual Gradient of loss w.r.t weight (dw): 6.691762447357178
Manual Gradient of loss w.r.t bias (db): 0.998770534992218


**Using Autograd**

In [27]:
x = torch.tensor(6.7)
y = torch.tensor(0.0)

In [28]:
w = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(0.0, requires_grad=True)

In [29]:
w

tensor(1., requires_grad=True)

In [30]:
b

tensor(0., requires_grad=True)

In [31]:
z = w*x + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [32]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [33]:
loss = binary_cross_entropy_loss(y_pred, y)
loss

tensor(6.7012, grad_fn=<NegBackward0>)

In [34]:
loss.backward()

In [35]:
print(w.grad)
print(b.grad)

tensor(6.6918)
tensor(0.9988)


**Some Operations using Autograd**

In [36]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

In [37]:
x

tensor([1., 2., 3.], requires_grad=True)

In [38]:
y = (x**2).mean()
y

tensor(4.6667, grad_fn=<MeanBackward0>)

In [39]:
y.backward()

In [40]:
x.grad

tensor([0.6667, 1.3333, 2.0000])

In [41]:
# clearing grad
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [42]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [43]:
y.backward()

In [44]:
x.grad

tensor(4.)

In [45]:
x.grad.zero_()

tensor(0.)

In [46]:
# disable gradient tracking
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [47]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

In [48]:
y.backward()

In [49]:
x.grad

tensor(4.)

In [50]:
# option 1 - requires_grad_(False)

In [51]:
x.requires_grad_(False)

tensor(2.)

In [52]:
x

tensor(2.)

In [53]:
y = x ** 2

In [54]:
y

tensor(4.)

In [55]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [56]:
# option 2 - detach()

In [57]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [58]:
z = x.detach()
z

tensor(2.)

In [59]:
y = x ** 2

In [60]:
y

tensor(4., grad_fn=<PowBackward0>)

In [61]:
y1 = z ** 2
y1

tensor(4.)

In [62]:
y.backward()

In [63]:
y1.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [64]:
# option 3 - torch.no_grad()

In [65]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [66]:
with torch.no_grad():
  y=x**2

In [67]:
y

tensor(4.)

In [68]:
y.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn